<div style="line-height:1.2;">

<h1 style="color:#0FA345; margin-bottom: 0.2em;">Random Forest Classification 1</h1>
<h4 style="margin-top: 0.2em; margin-bottom: 0.5em;"> Example with sklearn RandomForestClassifier on the iris dataset. Focus on metrics. </h4>

</div>

<div style="line-height:1.4; margin-top: 0.2em;">
<span>
    <h3 style="color: lightblue; display: inline; margin-right: 0.5em;">Keywords:</h3>
    sklearn ignore_warnings + RandomizedSearchCV + classification_report + Tensorboard with PyTorch + jaccard_score + hamming loss
</span>
</div>

In [2]:
%%script echo Skipping, since already installed
!pip install tensorboardX

Skipping, since already installed


In [32]:
import os
import joblib
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorboardX import SummaryWriter 
#from torch.utils.tensorboard import SummaryWriter

from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.datasets import make_multilabel_classification
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB

from sklearn.exceptions import ConvergenceWarning

from sklearn.model_selection import RandomizedSearchCV, validation_curve, StratifiedKFold, ShuffleSplit, cross_val_score
from sklearn.model_selection import train_test_split as tts

from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, hamming_loss, classification_report

from sklearn.metrics import zero_one_loss
from sklearn.metrics import precision_recall_fscore_support as scorep
from sklearn.metrics import jaccard_score


from yellowbrick.model_selection import ValidationCurve

try:
    from sklearn.utils._testing import ignore_warnings
except ImportError:
    from sklearn.utils.testing import ignore_warnings

In [4]:
location = "./data/iris_correct.csv"

In [5]:
my_dataset = pd.read_csv(location, skiprows=None, engine='python')
my_dataset.drop(columns=['species'], inplace=True)
my_dataset

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [6]:
%%script echo Skipping: The dataset it is already in its correct form.
my_dataset["sepal_width"] = [float(str(i).replace(",", "")) for i in my_dataset["sepal_width"]]
my_dataset["petal_length"] = [float(str(i).replace(",", "")) for i in my_dataset["petal_length"]]
my_dataset["sepal_length"] = [float(str(i).replace(",", "")) for i in my_dataset["sepal_length"]]
my_dataset["petal_width"] = [float(str(i).replace(",", "")) for i in my_dataset["petal_width"]]

Skipping: The dataset it is already in its correct form.


In [7]:
my_dataset = shuffle(my_dataset, random_state=3)
if 'variety' in my_dataset.columns:
    Y = my_dataset[['variety']]
    X = my_dataset
    X.pop('variety')

In [8]:
X.head(), X.shape

(    sepal.length  sepal.width  petal.length  petal.width
 47           4.6          3.2           1.4          0.2
 3            4.6          3.1           1.5          0.2
 31           5.4          3.4           1.5          0.4
 25           5.0          3.0           1.6          0.2
 15           5.7          4.4           1.5          0.4,
 (150, 4))

In [9]:
""" Split:
Train set correspond to the 75% of the original dataset
Test set is 10% of the initial dataset
Validation set is now 15% of the initial data set
"""
train_ratio = 0.80
validation_ratio = 0.10
test_ratio = 0.10
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=1 - train_ratio)

X_val, X_test, Y_val, Y_test = tts(X_test, Y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

In [10]:
print(Y_train, Y_val, Y_test)

        variety
132   Virginica
147   Virginica
62   Versicolor
143   Virginica
71   Versicolor
..          ...
122   Virginica
139   Virginica
134   Virginica
26       Setosa
44       Setosa

[120 rows x 1 columns]         variety
75   Versicolor
30       Setosa
77   Versicolor
20       Setosa
96   Versicolor
14       Setosa
27       Setosa
69   Versicolor
1        Setosa
149   Virginica
141   Virginica
97   Versicolor
84   Versicolor
87   Versicolor
92   Versicolor         variety
79   Versicolor
58   Versicolor
135   Virginica
83   Versicolor
148   Virginica
103   Virginica
42       Setosa
23       Setosa
19       Setosa
43       Setosa
126   Virginica
54   Versicolor
95   Versicolor
145   Virginica
119   Virginica


#### => RandomizedSearchCV

In [11]:
%%script echo Skipping due to overtime 

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}

####### Search for best hyperparameters with the random grid, using 3 fold cross-validation.
# Create the base model to tune
rf = RandomForestClassifier()
# Search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, 
                                verbose=2, random_state=42, n_jobs = -1)
# Fit the search model
rf_random.fit(X_train, Y_train)

Skipping due to overtime


In [12]:
%%script echo Skipping, since the RandomizedSearchCV was not completed.
rf_random.best_params_

Skipping, since the RandomizedSearchCV was not completed.


In [13]:
clf = RandomForestClassifier(n_estimators=1000, min_samples_split = 5,
                            min_samples_leaf=1, 
                            max_features='sqrt',
                            max_depth=30,class_weight="balanced",bootstrap = True)

In [14]:
# Change shape of targets' vector to test  
Y_train2 = np.ravel(Y_train)
Y_train2.shape, Y_train.shape

((120,), (120, 1))

In [15]:
clf.fit(X_train, np.ravel(Y_train)); #use the flattened array!

### => Metrics:

In [19]:
""" Cross validation.
Reshape target variables to 1D arrays with ravel()
=> To avoid the error : DataConversionWarning: A column-vector y was passed when a 1d array was expected. 
"""
Y_train = np.ravel(Y_train)
Y_test = np.ravel(Y_test)
Y_val = np.ravel(Y_val)

scores = clf.score(X_train, Y_train)
scores_test = clf.score(X_test, Y_test)
cross_scores_tra = cross_val_score(clf, X_train, Y_train, cv=5) 
cross_scores_val = cross_val_score(clf, X_val, Y_val, cv=5)
cross_scores_test = cross_val_score(clf, X_test, Y_test, cv=5)

print("SCORE:")
print(scores)
print("SCORE test:")
print(scores_test)

print("CROSSSCORE val:")
print(cross_scores_val)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_val.mean(), cross_scores_val.std()))
print("CROSSSCORE tra:")
print(cross_scores_tra)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_tra.mean(), cross_scores_tra.std()))
print("CROSSSCORE test:")
print(cross_scores_test)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_test.mean(), cross_scores_test.std()))

print(f"How many coumns ins my_dataset? {len(my_dataset.columns)}")

/home/notto4/anaconda3/envs/MLearning/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/notto4/anaconda3/envs/MLearning/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


SCORE:
0.9916666666666667
SCORE test:
0.8666666666666667
CROSSSCORE val:
[0.66666667 1.         1.         0.66666667 1.        ]
0.87 accuracy with a standard deviation of 0.16
CROSSSCORE tra:
[0.95833333 0.91666667 0.95833333 0.95833333 1.        ]
0.96 accuracy with a standard deviation of 0.03
CROSSSCORE test:
[1.         1.         0.66666667 1.         0.66666667]
0.87 accuracy with a standard deviation of 0.16
How many coumns ins my_dataset? 4


The warning above indicates indicates that one or more of the classes have very few instances. <br>
This can potentially cause issues when performing cross-validation with a high number of splits. <br>
One solution to address the problem can be to use Stratified cross-validation.

In [20]:
""" Stratified Sampling.
To ensure that each fold of the cross-validation contains a proportional representation of each class
"""
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Use stratified cross-validation
cross_scores_tra = cross_val_score(clf, X_train, Y_train, cv=cv)
cross_scores_test = cross_val_score(clf, X_test, Y_test, cv=cv)
cross_scores_val = cross_val_score(clf, X_val, Y_val, cv=cv)

print("CROSSSCORE val:")
print(cross_scores_val)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_val.mean(), cross_scores_val.std()))
print("CROSSSCORE tra:")
print(cross_scores_tra)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_tra.mean(), cross_scores_tra.std()))
print("CROSSSCORE test:")
print(cross_scores_test)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_test.mean(), cross_scores_test.std()))

/home/notto4/anaconda3/envs/MLearning/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/home/notto4/anaconda3/envs/MLearning/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


CROSSSCORE val:
[1.         1.         0.66666667 1.         0.66666667]
0.87 accuracy with a standard deviation of 0.16
CROSSSCORE tra:
[1.         1.         0.95833333 0.95833333 0.875     ]
0.96 accuracy with a standard deviation of 0.05
CROSSSCORE test:
[1.         1.         0.66666667 0.66666667 1.        ]
0.87 accuracy with a standard deviation of 0.16


ShuffleSplit (another Cross-Validation Strategy) to generates random subsets of data for each split. <br>
Useful when dealing with imbalanced datasets.

In [21]:
""" Reduce the splits to 3 """
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
cross_scores_tra = cross_val_score(clf, X_train, Y_train, cv=cv)
cross_scores_test = cross_val_score(clf, X_test, Y_test, cv=cv)
cross_scores_val = cross_val_score(clf, X_val, Y_val, cv=cv)

print("CROSSSCORE val:")
print(cross_scores_val)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_val.mean(), cross_scores_val.std()))
print("CROSSSCORE tra:")
print(cross_scores_tra)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_tra.mean(), cross_scores_tra.std()))
print("CROSSSCORE test:")
print(cross_scores_test)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cross_scores_test.mean(), cross_scores_test.std()))

CROSSSCORE val:
[0.66666667 1.         1.         0.66666667 1.        ]
0.87 accuracy with a standard deviation of 0.16
CROSSSCORE tra:
[0.95833333 1.         1.         0.95833333 0.95833333]
0.97 accuracy with a standard deviation of 0.02
CROSSSCORE test:
[1. 1. 1. 1. 1.]
1.00 accuracy with a standard deviation of 0.00


In [27]:
""" 
NB in case multi_target columns --> ValueError: multiclass-multioutput is not supported !
The MultiOutputClassifier does not support multi-class multi-output classification. 
It is designed for multi-output regression problems, where each output is a continuous variable.
"""
yhat = clf.predict(X_test)
cr_y1 = classification_report(Y_test,yhat)
print(cr_y1)

              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00         4
  Versicolor       0.80      0.80      0.80         5
   Virginica       0.83      0.83      0.83         6

    accuracy                           0.87        15
   macro avg       0.88      0.88      0.88        15
weighted avg       0.87      0.87      0.87        15



### => Other Metrics:

In [28]:
zer = zero_one_loss(Y_test, yhat,normalize=False)
zer

2

In [29]:
jac = jaccard_score(Y_test, yhat, average="micro")
jac

0.7647058823529411

In [30]:
hamm = hamming_loss(Y_test, yhat)
hamm

0.13333333333333333

In [33]:
precision, recall, fscore, support = scorep(Y_test, yhat)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [1.         0.8        0.83333333]
recall: [1.         0.8        0.83333333]
fscore: [1.         0.8        0.83333333]
support: [4 5 6]


In [34]:
#preci = precision_score(Y_test.iloc[:, 0], yhat[:, 0], average='micro')
preci = precision_score(Y_test, yhat, average='micro')
preci

0.8666666666666667

## Tensorboard

In [35]:
# Create a SummaryWriter object to log data to TensorBoard
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
writer = SummaryWriter(log_dir)

In [36]:
# Log each value separately with a unique tag
for idx, score in enumerate(cross_scores_val):
    tag = f"CROSSSCORE val {idx}"
    writer.add_scalar(tag, score)

In [37]:
jac = jaccard_score(Y_test, yhat, average="micro")
zer = zero_one_loss(Y_test, yhat,normalize=False)
hamm = hamming_loss(Y_test, yhat)

writer.add_scalar('jaccard_score test', jac)
writer.add_scalar('zero_one_loss test', zer)
writer.add_scalar('hamming_loss test', hamm)

In [40]:
for idx, prec in enumerate(precision):
    tag = f"Precision {idx}"
    writer.add_scalar(tag, prec)

for idx, reca in enumerate(recall):
    tag = f"Recall {idx}"
    writer.add_scalar(tag, reca)

for idx, scor in enumerate(fscore):
    tag = f"F1score {idx}"
    writer.add_scalar(tag, scor)

for idx, supp in enumerate(support):
    tag = f"Precision {idx}"
    writer.add_scalar(tag, supp)            

In [43]:
writer.close()

In [44]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 70790), started 0:14:33 ago. (Use '!kill 70790' to kill it.)